## **ASSIGNMENT 3 PART 1**

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, Dense
from sklearn.model_selection import train_test_split
import pandas as pd
# Load dataset
df = pd.read_csv('IMDB Dataset.csv')

#Prepare data
texts = df['review'].values
labels = df['sentiment'].map({'positive': 1, 'negative': 0}).values

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Tokenization
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Padding sequences
maxlen = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=maxlen)
X_test_padded = pad_sequences(X_test_sequences, maxlen=maxlen)


In [ ]:
# Download the GloVe embeddings (in your terminal or Jupyter notebook cell)
!wget http://nlp.stanford.edu/data/glove.6B.zip

# Unzip the file
!unzip glove.6B.zip


--2024-09-01 06:25:32--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-09-01 06:25:32--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-09-01 06:25:32--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [28]:
import numpy as np

# Load GloVe embeddings
def load_glove_embeddings(file_path):
    embedding_index = {}
    with open(file_path, 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = coefs
    return embedding_index

glove_file_path = 'glove.6B.100d.txt'
embedding_index = load_glove_embeddings(glove_file_path)

# Create an embedding matrix
embedding_dim = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in embedding_index:
        embedding_matrix[i] = embedding_index[word]


In [31]:
# Create embedding matrix
embedding_dim = 100
word_index = tokenizer.word_index
num_words = min(10000, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))

# Make sure to execute the cell where 'embedding_index' is defined before running this cell
for word, i in word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embedding_index.get(word) # Now you can access 'embedding_index'
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [32]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(SimpleRNN(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_data=(X_test_padded, y_test))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 24s 34ms/step - accuracy: 0.6143 - loss: 0.6471 - val_accuracy: 0.6846 - val_loss: 0.5965
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 45s 41ms/step - accuracy: 0.6638 - loss: 0.6093 - val_accuracy: 0.6926 - val_loss: 0.5973
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 25s 41ms/step - accuracy: 0.6494 - loss: 0.6243 - val_accuracy: 0.6046 - val_loss: 0.6569
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.6669 - loss: 0.6104 - val_accuracy: 0.6938 - val_loss: 0.6535
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 43s 37ms/step - accuracy: 0.6569 - loss: 0.6249 - val_accuracy: 0.7074 - val_loss: 0.5776


In [33]:
from keras.layers import LSTM

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_data=(X_test_padded, y_test))


Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 131ms/step - accuracy: 0.6833 - loss: 0.5725 - val_accuracy: 0.7935 - val_loss: 0.4395
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 128ms/step - accuracy: 0.8128 - loss: 0.4057 - val_accuracy: 0.8241 - val_loss: 0.3872
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 126ms/step - accuracy: 0.8419 - loss: 0.3579 - val_accuracy: 0.8434 - val_loss: 0.3472
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 132ms/step - accuracy: 0.8516 - loss: 0.3345 - val_accuracy: 0.8508 - val_loss: 0.3320
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 140s 128ms/step - accuracy: 0.8656 - loss: 0.3111 - val_accuracy: 0.8510 - val_loss: 0.3287


In [34]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
from torch import nn, optim

# Convert to tensor
X_train_tensor = torch.tensor(X_train_padded, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_padded, dtype=torch.long)
y_train_tensor = torch.tensor(y_train, dtype=torch.float)
y_test_tensor = torch.tensor(y_test, dtype=torch.float)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)


In [35]:
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(RNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])
        return torch.sigmoid(x)

model = RNNModel(num_words, embedding_dim, 64)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_loader:
            inputs, labels = batch
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Epoch {epoch + 1}, Accuracy: {correct / total}')


Epoch 1, Accuracy: 0.7358
Epoch 2, Accuracy: 0.7559
Epoch 3, Accuracy: 0.7725
Epoch 4, Accuracy: 0.6792
Epoch 5, Accuracy: 0.7736


In [36]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return torch.sigmoid(x)

model = LSTMModel(num_words, embedding_dim, 64)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    model.train()
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_loader:
            inputs, labels = batch
            outputs = model(inputs)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print(f'Epoch {epoch + 1}, Accuracy: {correct / total}')


Epoch 1, Accuracy: 0.8086
Epoch 2, Accuracy: 0.8443
Epoch 3, Accuracy: 0.8637
Epoch 4, Accuracy: 0.864
Epoch 5, Accuracy: 0.871
